# Evolution of a Crawling Behavior for a Humanoid Robot Using Genetic Algorithms

<img src="files/img/crawling.png" />

Controlling a biped robot with several degrees of freedom is a challenging task. For a humanoid robot to perform in complex environments, fast, stable and adaptive behaviors are required. In this session you will work on a solution for automatic generation of a [crawling](http://en.wikipedia.org/wiki/Crawling_(human&#41;) gait using genetic algorithms (GA).

## Aim of this notebook

You will use Pyevolve for tuning the parameters of a crawling controller of a simulated [NAO robot](https://www.softbankrobotics.com/emea/es/nao). You will perform experiments to validate the performance of the evolution strategy.

## The Crawling Gait

Some human-like movements are inherently periodic and repeat the same set of steps several times (e.g. walk, crawl, etc). Such a periodic function can be decomposed into a sum of simple oscillators as represented by the following expression: 

<img src="files/img/eq1.png" width="480" />

where $N$ is the number of frequencies, $C$ is the offset, $A_{n=1..N}$ are amplitudes, $T$ is the period and $Φ_{n=1..N}$ are phases.
Applying these oscillators to each joint, a crawling gait can be developed and tested with the simulated humanoid NAO.

<img src="files/img/joints.png" />

The figure shows the humanoid structure and the referential axis considered ([more details about NAO joints can be found here](http://doc.aldebaran.com/1-14/family/robots/joints_robot.html)). The main idea behind the definition of this gait is to place an oscillator on each joint we pretend to move in order to define its trajectory. The oscillators are placed on the following joints of the left (L) and right (R) sides: ShoulderPitch, ShoulderRoll, ElbowRoll HipPitch, HipRoll, and KneePitch (note: the "hip" is referred to as "thigh" in the figure).

Therefore, 12 single-frequency oscillators are used. Since each single-frequency oscillator will have 4 parameters to define, 48 parameters are needed to completely define the gait. It is common to assume a sagittal symmetry, which determines the same movements for corresponding left and right sided joints with a half-period phase shift. Hence, it is possible to reduce the number of parameters by half of the original size, resulting on 24 parameters. Additionally, the period of all oscillators should be the same to keep all the joints synchronized by a single frequency clock. This consideration reduces the number of parameters to 19. If the parameters are defined on the left-sided joints, the right-sided joints can be readily obtained: for roll joints the left and the right side perform the same trajectories over the time; only the sign of the offset needs to be changed. For pitch joints, the right side can be obtained by adding a phase, π, to the corresponding oscillator. The unknown parameters together form the genome that will be used by the genetic algorithm to generate the gait. 

## Simulation setup

Run webots and open the world `GA-robot/worlds/nao_demo.wbt`.

## Connecting to the NAO robot

Once the simulator is running and the robot has been spawned, you can proceed with the following statements:

In [ ]:
from Nao import Nao

Creating a `nao` object from class `Nao`:

In [ ]:
nao = Nao()

Setting the crawling posture:

In [ ]:
nao.initCrawling()

Run a crawling cycle:

In [ ]:
import math
params = [1.28,0.1,0.21,0.,0.035,0.039,-2.,0.12,-0.86,math.pi,\
          0.06,0.33,math.pi/2,0.005,-0.11,-2.,0.008,1.8,0.]
nao.crawl(params, seconds=5)

The result is the distance travelled by the robot during the specified time in seconds.

The argument `params` is a list with the period, and the amplitude (A), offset (K) and phase (Phi) of each joint oscillator:

* period
* shoulderPitchA
* shoulderPitchK
* shoulderPitchPhi
* shoulderRollA
* shoulderRollK
* shoulderRollPhi
* hipPitchA
* hipPitchK
* hipPitchPhi
* hipRollA
* hipRollK
* hipRollPhi
* elbowRollA
* elbowRollK
* elbowRollPhi
* kneePitchA
* kneePitchK
* kneePitchPhi

This video shows the resulting behavior in the simulated Nao: [https://youtu.be/Yq_IFlMAEvM](https://youtu.be/Yq_IFlMAEvM)

## Task 1: GA Configuration

You should develop a Pyevolve application for the crawling problem, with the following restrictions:

* Use a 1D list chromosome (see http://pyevolve.sourceforge.net/0_6rc1/module_g1dlist.html)
* An initial population of 10 chromosomes initialized randomly
* Roulette method for selection (use [ga.selector.set](http://pyevolve.sourceforge.net/0_6rc1/module_selectors.html))
* Real range method for mutation (use genome.mutator.set), with a probability defined by pm=0.5 (use function setMutationRate)
* Uniform method for crossover (use [genome.crossover.set](http://pyevolve.sourceforge.net/0_6rc1/module_crossovers.html)
* Fraction of the population created by crossover defined by pc=0.8 (use function setCrossoverRate)

The fitness function has to be chosen carefully in order to achieve good results. A simple but effective fitness function to maximize can be the travelled distance during a fixed time.

The 1D list chromosome accepts the `rangemin` and `rangemax` parameters, but they are shared by all the elements of the genome. You should define a standard range (e.g. [0,1]) and then scale each element to any other pre-defined range, prior to sending the crawl motion to the robot. Scaling is easily calculated by the expression:

    robot_parameter = genome_value * (max - min) + min

The suggested ranges for each joint parameter are shown in the following table:

<table>
<tr>
<th>&nbsp;</th>
<th>min</th>
<th>max</th>
</tr>
<tr>
<td height="22" align="RIGHT">period</td>
<td align="CENTER" sdval="0,8" sdnum="1027;">0,8</td>
<td align="CENTER" sdval="1,8" sdnum="1027;">1,8</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderPitchA</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,5" sdnum="1027;">0,5</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderPitchK</td>
<td align="CENTER" sdval="-0,5" sdnum="1027;">-0,5</td>
<td align="CENTER" sdval="0,5" sdnum="1027;">0,5</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderPitchPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderRollA</td>
<td align="CENTER" sdval="0,2" sdnum="1027;">0,2</td>
<td align="CENTER" sdval="0,4" sdnum="1027;">0,4</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderRollK</td>
<td align="CENTER" sdval="-0,1" sdnum="1027;">-0,1</td>
<td align="CENTER" sdval="0,1" sdnum="1027;">0,1</td>
</tr>
<tr>
<td height="22" align="RIGHT">shoulderRollPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipPitchA</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,4" sdnum="1027;">0,4</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipPitchK</td>
<td align="CENTER" sdval="-1,2" sdnum="1027;">-1,2</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipPitchPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipRollA</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,2" sdnum="1027;">0,2</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipRollK</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,5" sdnum="1027;">0,5</td>
</tr>
<tr>
<td height="22" align="RIGHT">hipRollPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>
<tr>
<td height="22" align="RIGHT">elbowRollA</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,1" sdnum="1027;">0,1</td>
</tr>
<tr>
<td height="22" align="RIGHT">elbowRollK</td>
<td align="CENTER" sdval="-0,5" sdnum="1027;">-0,5</td>
<td align="CENTER" sdval="-0,2" sdnum="1027;">-0,2</td>
</tr>
<tr>
<td height="22" align="RIGHT">elbowRollPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>
<tr>
<td height="22" align="RIGHT">kneePitchA</td>
<td align="CENTER" sdval="0" sdnum="1027;">0</td>
<td align="CENTER" sdval="0,2" sdnum="1027;">0,2</td>
</tr>
<tr>
<td height="22" align="RIGHT">kneePitchK</td>
<td align="CENTER" sdval="1,6" sdnum="1027;">1,6</td>
<td align="CENTER" sdval="1,9" sdnum="1027;">1,9</td>
</tr>
<tr>
<td height="22" align="RIGHT">kneePitchPhi</td>
<td align="CENTER" sdval="-3,14" sdnum="1027;">-3,14</td>
<td align="CENTER" sdval="3,14" sdnum="1027;">3,14</td>
</tr>

</table>

You are encouraged to experiment with your own ranges, but you should take into account the physical limits of the robot joints ([see the Nao documentation](http://doc.aldebaran.com/1-14/family/nao_h21/joints_h21_v32.html)).

## Task 2: Experimental Results

Execute your GA and analyze the results:

1. Include the necessary statements for storing the results in a database named 'crawling.db' with identifier 'ex1' (50 points).
2. For 15 extra points, the evolution process should reach 20 generations, with a population of at least 10 individuals
3. For other 15 extra points (maximum grade), the evolution process should reach 50 generations, with a population of at least 10 individuals

You can check now the results by plotting some graphs of the evolution process in [this notebook](Crawling_check.ipynb).